In [25]:
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
# Get the prompt to use 
# https://github.com/hwchase17/langchain-hub?tab=readme-ov-file
#prompt = load_prompt('lc://prompts/conversation/prompt.json')

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(
            variable_name="chat_history",optional=True
        ),
        ("human", "{input}"),
        MessagesPlaceholder(
            variable_name="agent_scratchpad",
        ),
    ]
)
prompt.messages
#prompt.pretty_print()




Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [32]:
from langchain import hub

instructions = """You are an assistant.
Please answer in a structured, clear, coherent, and comprehensive manner.
If the content has sections, please summarize them in order and present them easy to read format.
if you are using table or figure number then make sure to add reference at the footnote.
please mention reference sources.
Please make sure to provide the answer in a structured, clear, coherent, and comprehensive manner.
"""

base_prompt = hub.pull("hwchase17/react")
react_prompt = base_prompt.partial(instructions=instructions)


print(react_prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] partial_variables={'instructions': 'You are an assistant.\nPlease answer in a structured, clear, coherent, and comprehensive manner.\nIf the content has sections, please summarize them in order and present them easy to read format.\nif you are using table or figure number then make sure to add reference at the footnote.\nplease mention reference sources.\nPlease make sure to provide the answer in a structured, clear, coherent, and comprehensive manner.\n'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know th

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.chains import LLMChain
from langchain.agents import AgentExecutor, create_openai_functions_agent, Tool


import os
from dotenv import load_dotenv

load_dotenv(override=True)
#os.environ["OPENAI_API_KEY"] = "your key"


@tool
def build(commit_id: int):
    """build artifact with commit_id ."""
    print(f"Building artifact with commit_id {commit_id}")
    build_id = 100
    return build_id

@tool
def deploy(build_id: int):
    """deploy artifact with build_id. """
    print(f"Deploying artifact with build_id {build_id}")
    deploy_id = 200
    return deploy_id

@tool
def test(deploy_id: int):
    "test artifact with deploy_id."
    print(f"Testing artifact with deploy_id {deploy_id}")
    test_id = 300
    return test_id

@tool
def report(test_id: int):
    "generate report with test_id."
    print(f"Generating report with test_id {test_id}")
    report_id = 400
    return report_id

#build.invoke({"commit_id": 1})

from operator import itemgetter
from typing import Union

from langchain.output_parsers import JsonOutputToolsParser
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-4-0125-preview")
llm_chain = LLMChain(llm=llm, prompt=prompt)
tools = [build, deploy, test, report, 
         Tool(
             name="reason",
             func=llm_chain.invoke,
             description = "assistant to help you with the reason",
         ),]



In [21]:
from langchain.agents import AgentExecutor

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": "build 256 and generate report"
    }
)



> Entering new AgentExecutor chain...
CXL can refer to different things depending on the context. Here are a few possibilities:

1. **Compute Express Link (CXL):** This is perhaps the most prominent reference in the field of computing and technology. Compute Express Link (CXL) is an open industry standard interconnect offering high-speed, efficient performance for data centers and computing environments. It's designed to improve the performance of data centers by enabling high-speed, efficient communication between the CPU (Central Processing Unit) and workload accelerators, memory buffers, and smart I/O devices. CXL is significant for its support of heterogenous computing and its ability to maintain memory coherency among devices. This makes it particularly useful for applications in AI, machine learning, and high-performance computing.

2. **Customer Experience Lifecycle (CXL):** In marketing and customer service, CXL often refers to the Customer Experience Lifecycle. This concept 

{'input': 'what is CXL',
 'output': 'CXL can refer to different things depending on the context. Here are a few possibilities:\n\n1. **Compute Express Link (CXL):** This is perhaps the most prominent reference in the field of computing and technology. Compute Express Link (CXL) is an open industry standard interconnect offering high-speed, efficient performance for data centers and computing environments. It\'s designed to improve the performance of data centers by enabling high-speed, efficient communication between the CPU (Central Processing Unit) and workload accelerators, memory buffers, and smart I/O devices. CXL is significant for its support of heterogenous computing and its ability to maintain memory coherency among devices. This makes it particularly useful for applications in AI, machine learning, and high-performance computing.\n\n2. **Customer Experience Lifecycle (CXL):** In marketing and customer service, CXL often refers to the Customer Experience Lifecycle. This concep